# Limpieza inicial de datasets

En este notebook se realiza la preparación básica de los datasets antes de su
transformación al modelo documental.

El objetivo no es limpiar completamente los datos ni eliminar valores nulos,
sino normalizar su formato para permitir su posterior integración en MongoDB.

Se aplican las siguientes transformaciones:

- Eliminación de espacios en blanco en columnas de texto
- Conversión de fechas a tipo datetime
- Normalización de tipos de datos
- Exportación a versión limpia (clean)


In [ ]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)

## Problemas detectados en los datasets originales

Durante la exploración inicial se observaron varios problemas comunes en datos
procedentes de sistemas administrativos:

- Espacios en blanco al inicio o final de los textos
- Fechas almacenadas como cadenas de texto
- Columnas con tipos mezclados (número y texto)
- Valores vacíos representados de distintas formas

Estos problemas no impiden la lectura del CSV, pero sí pueden provocar:

- Errores al generar JSON
- Tipos inconsistentes en MongoDB
- Resultados incorrectos en agregaciones

Por ello se aplica una normalización ligera previa al modelado.


In [ ]:
def limpiar_strings(df):
    """Elimina espacios en blanco al inicio y final de todos los textos"""
    for col in df.select_dtypes(include=["object", "string"]).columns:
        df[col] = df[col].astype("string").str.strip()
        df[col] = df[col].replace("", pd.NA)
    return df


def convertir_fechas(df):
    """Convierte automáticamente columnas con fecha"""
    posibles = [c for c in df.columns if "fecha" in c.lower() or "fx_" in c.lower()]
    
    for col in posibles:
        try:
            df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)
        except:
            pass
    return df


def normalizar_tipos(df):
    """Evita tipos mezclados que rompen Mongo"""
    for col in df.columns:
        if df[col].dtype == "object":
            # intenta convertir a numérico si parece número
            convertido = pd.to_numeric(df[col], errors="ignore")
            df[col] = convertido
    return df

def limpiar_dataset(path):
    df = pd.read_csv(path, sep=';', low_memory=False)
    
    print("Cargando:", path)
    print("Shape original:", df.shape)
    
    df = limpiar_strings(df)
    df = convertir_fechas(df)
    df = normalizar_tipos(df)
  
    print("Limpieza completada\n")
    return df
    



## Carga y limpieza de los datasets

Cada fichero se carga individualmente y se somete al mismo proceso de
normalización para garantizar coherencia entre todos los conjuntos de datos.

In [ ]:
df_terrazas = limpiar_dataset("../../data/raw/terrazas202312.csv")
df_locales = limpiar_dataset("../../data/raw/locales202312.csv")
df_licencias = limpiar_dataset("../../data/raw/licencias202312.csv")
df_actividad = limpiar_dataset("../../data/raw/actividadeconomica202312.csv")

## Verificación de la limpieza

Se inspecciona la estructura resultante para comprobar:

- Tipos de datos correctos
- Ausencia de espacios residuales
- Conversión adecuada de fechas


In [ ]:
df_terrazas.head()

In [ ]:
df_terrazas.info()

## Exportación de datasets normalizados

Los datasets resultantes se guardan en una versión *clean*.

Estos archivos no contienen cambios estructurales ni pérdida de información;
únicamente se ha normalizado su formato para permitir su posterior
transformación al modelo documental y su inserción en MongoDB.

In [ ]:
df_terrazas.to_csv("../../data/cleaned/terrazas_clean.csv", index=False)
df_locales.to_csv("../../data/cleaned/locales_clean.csv", index=False)
df_licencias.to_csv("../../data/cleaned/licencias_clean.csv", index=False)
df_actividad.to_csv("../../data/cleaned/actividad_clean.csv", index=False)

print("Datasets limpios guardados en /data/clean")
